In [1]:
%matplotlib inline
import torch
from torch import nn,optim
from torch.utils import data

In [2]:
def load_array(features,batch_size, resize=None):  #@save
    dataset = data.TensorDataset(torch.tensor(features[0],dtype=torch.float32),torch.tensor(features[1],dtype=torch.float32))
    return data.DataLoader(dataset, batch_size, shuffle=True,
                            num_workers=12)
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))
n_train, n_test, num_inputs, batch_size = 20, 100, 200, 5
true_w, true_b = torch.ones((num_inputs, 1)) * 0.01, 0.05
train_data = synthetic_data(true_w, true_b, n_train)
train_iter = load_array(train_data, batch_size)
test_data = synthetic_data(true_w, true_b, n_test)
test_iter = load_array(test_data, batch_size)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19164\1478305030.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = data.TensorDataset(torch.tensor(features[0],dtype=torch.float32),torch.tensor(features[1],dtype=torch.float32))


In [3]:

def l2_penalty(w):
    return torch.sum(w.pow(2)) / 2

In [ ]:
def train(lambd):

    net, loss = nn.Sequential(nn.Linear(num_inputs,1)),torch.nn.MSELoss()
    num_epochs, lr = 100, 0.003
    net[0].weight.data.normal_(0,0.01)
    net[0].bias.data.zero_()
    updater=torch.optim.SGD( [{"params":net[0].weight,'weight_decay': lambd},
        {"params":net[0].bias}], lr)
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            # 增加了L2范数惩罚项，
            # 广播机制使l2_penalty(w)成为一个长度为batch_size的向量
            l = loss(net(X), y) + lambd * l2_penalty(net[0].weight)
            updater.zero_grad()
            l.backward()
            updater.step()
        if (epoch + 1) % 3 == 0:
            print("train：",loss(net(train_data[0]),train_data[1]).mean())
            print("test：",loss(net(test_data[0]),test_data[1]).mean())
    print('w的L2范数是：', torch.norm(net[0].weight).item())
train(3)

训练： tensor(0.0080, grad_fn=<MeanBackward0>)
测试： tensor(0.0310, grad_fn=<MeanBackward0>)
训练： tensor(0.0025, grad_fn=<MeanBackward0>)
测试： tensor(0.0274, grad_fn=<MeanBackward0>)
训练： tensor(0.0014, grad_fn=<MeanBackward0>)
测试： tensor(0.0250, grad_fn=<MeanBackward0>)
训练： tensor(0.0011, grad_fn=<MeanBackward0>)
测试： tensor(0.0235, grad_fn=<MeanBackward0>)
训练： tensor(0.0010, grad_fn=<MeanBackward0>)
测试： tensor(0.0224, grad_fn=<MeanBackward0>)
训练： tensor(0.0010, grad_fn=<MeanBackward0>)
测试： tensor(0.0217, grad_fn=<MeanBackward0>)


KeyboardInterrupt: 